In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/master_thesis2')

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 27.1 MB/s eta 0:00:00
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-sci-sm 0.5.1 requires spacy<3.5.0,>=3.4.1, but you have spacy 3.7.5 which is incompatible.
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz (15.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
  Usi

1. Build the reference corpus

In [ ]:
!pip install rdflib

In [ ]:
import os
import re
import numpy as np
import spacy
import networkx as nx

from tqdm import tqdm
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import chi2_contingency

# For RDF output
from rdflib import Graph as RDFGraph, Namespace, URIRef, RDF
from rdflib.namespace import RDFS
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

# 1) LOAD SCISPACY MODEL (WITH UMLS LINKER)
def load_scispacy_model_with_linker(threshold=0.8):
    """
    Loads a SciSpaCy model (en_core_sci_sm) and adds:
      - AbbreviationDetector
      - scispacy_linker for UMLS with the specified threshold.
    """
    print("Loading SciSpaCy model...")
    nlp = spacy.load("en_core_sci_sm")
    print("Initial pipeline components:", nlp.pipe_names)

    nlp.add_pipe("abbreviation_detector", last=True)
    nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
    return nlp

# 2. TEXT NORMALIZATION
def normalize_text(term):
    """
    Normalize term by removing underscores and ensuring consistent capitalization.
    """
    term = term.replace("_", " ").strip()
    term = " ".join(word.capitalize() for word in term.split())
    return term

# 3. CONTEXTUAL DISAMBIGUATION
def disambiguate_entity(entity, context, linker):
    """
    Disambiguate an entity using its surrounding context and UMLS linking.
    """
    CANCER_SEMTYPES = {"T191"}  # Neoplastic Processes
    if entity._.kb_ents:
        for cui, score in entity._.kb_ents:
            concept = linker.kb.cui_to_entity[cui]
            if CANCER_SEMTYPES.intersection(concept.types) and "breast" in context.lower():
                return concept
    return None

# 4. EXTRACT CANCER ENTITIES
def extract_cancer_entities(text, nlp):
    """
    Extracts entities related to cancer (T191) using SciSpaCy's EntityLinker.
    """
    doc = nlp(text)
    linker = nlp.get_pipe("scispacy_linker")

    cancer_entities = []
    for ent in doc.ents:
        context = text
        concept = disambiguate_entity(ent, context, linker)
        if concept:
            term = normalize_text(ent.text)
            cancer_entities.append(term)

    return cancer_entities

# 5. CHI-SQUARE SIGNIFICANCE
def calculate_statistical_significance(domain_freq, ref_freq, total_domain_terms, total_ref_terms):
    """
    Calculates the chi-square p-value for term frequency significance.
    """
    table = [
        [domain_freq, ref_freq],
        [total_domain_terms - domain_freq, total_ref_terms - ref_freq]
    ]
    _, p_val, _, _ = chi2_contingency(table)
    return p_val

# 6. TAXONOMY BUILDING
def initialize_taxonomy():
    """
    Initializes a taxonomy graph with a root node "Breast Cancer".
    """
    G = nx.DiGraph()
    G.add_node("Breast Cancer")
    return G

def insert_concept_recursive(G, concept, descriptions, threshold=0.5, start_node="Breast Cancer", depth_limit=50, current_depth=0):
    """
    Inserts a concept under the most semantically similar child node.
    """
    if current_depth >= depth_limit:
        print(f"Depth limit reached for concept '{concept}'. Adding under '{start_node}'.")
        G.add_edge(start_node, concept)
        return

    children = list(G.successors(start_node))
    if not children:
        G.add_edge(start_node, concept)
        return

    concept_emb = descriptions[concept].reshape(1, -1)
    child_embs = np.array([descriptions[child] for child in children])
    sims = cosine_similarity(concept_emb, child_embs)[0]
    max_sim_idx = np.argmax(sims)
    max_sim = sims[max_sim_idx]

    if max_sim > threshold:
        next_node = children[max_sim_idx]
        insert_concept_recursive(G, concept, descriptions, threshold, next_node, depth_limit, current_depth + 1)
    else:
        G.add_edge(start_node, concept)

# 7. TAXONOMY POSTPROCESSING
def clean_ambiguous_nodes(taxonomy, descriptions, threshold=0.8):
    """
    Merge or reassign nodes with ambiguous or redundant meanings.
    """
    nodes = list(taxonomy.nodes())
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i + 1:]:
            if node1 != node2 and cosine_similarity([descriptions[node1]], [descriptions[node2]])[0][0] > threshold:
                nx.relabel_nodes(taxonomy, {node2: node1}, copy=False)

def remove_singleton_nodes(taxonomy):
    """
    Remove singleton nodes that have no meaningful connections.
    """
    for node in list(taxonomy.nodes()):
        if taxonomy.degree(node) == 0:
            taxonomy.remove_node(node)

# 8. EXPORT TAXONOMY TO RDF
def export_taxonomy_to_rdf(nx_graph, output_path):
    """
    Exports the taxonomy to RDF format.
    """
    rdf_graph = RDFGraph()
    TAXO = Namespace("http://example.org/taxonomy#")

    def safe(node):
        return re.sub(r"[^a-zA-Z0-9_-]", "_", node)

    for node in nx_graph.nodes():
        node_uri = TAXO[safe(node)]
        rdf_graph.add((node_uri, RDF.type, RDFS.Class))

    for parent, child in nx_graph.edges():
        parent_uri = TAXO[safe(parent)]
        child_uri = TAXO[safe(child)]
        rdf_graph.add((child_uri, RDFS.subClassOf, parent_uri))

    rdf_graph.serialize(destination=output_path, format='xml')
    print(f"RDF Taxonomy saved to {output_path}")


In [ ]:
# MAIN PIPELINE

# Load SciSpaCy model
nlp = load_scispacy_model_with_linker(threshold=0.8)

# Load SentenceTransformer
print("Loading SentenceTransformer...")
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Model loaded.\n")

# Load domain and reference corpora
domain_corpus_path = "/content/drive/MyDrive/master_thesis2/data/pubmed/pubmed_cancer_corpus.txt"
reference_corpus_path = "/content/drive/MyDrive/master_thesis2/statistical_method/data/reference_corpus.txt"

with open(domain_corpus_path, "r") as f:
  domain_corpus = f.readlines()
with open(reference_corpus_path, "r") as f:
  reference_corpus = f.readlines()

# Extract cancer entities from domain corpus
print("Extracting cancer entities from domain corpus...")
domain_cancer_entities = []
for text in tqdm(domain_corpus, desc="Domain Corpus"):
  domain_cancer_entities.extend(extract_cancer_entities(text, nlp))

# Compute reference term frequencies
print("Computing term frequencies for reference corpus...")
reference_term_freq = Counter()
for text in tqdm(reference_corpus, desc="Reference Corpus"):
  reference_term_freq.update(text.split())

# Compute domain term frequencies
domain_term_freq = Counter(domain_cancer_entities)
total_domain_terms = sum(domain_term_freq.values())
total_ref_terms = sum(reference_term_freq.values())

# Filter by chi-square significance
print("Identifying domain-specific concepts...")
new_concepts = []
for term, d_freq in domain_term_freq.items():
  r_freq = reference_term_freq.get(term, 0)
  if calculate_statistical_significance(d_freq, r_freq, total_domain_terms, total_ref_terms) < 0.05:
      new_concepts.append(term)

# Compute embeddings
print("Computing embeddings...")
descriptions = {term: model.encode([term])[0] for term in new_concepts}

# Build taxonomy
taxonomy = initialize_taxonomy()
print("Building taxonomy...")
for concept in new_concepts:
  insert_concept_recursive(taxonomy, concept, descriptions, threshold=0.5)

# Postprocessing
print("Postprocessing taxonomy...")
clean_ambiguous_nodes(taxonomy, descriptions, threshold=0.8)
remove_singleton_nodes(taxonomy)

# Export to RDF
output_path = "taxonomy.rdf"
export_taxonomy_to_rdf(taxonomy, output_path)

Loading SciSpaCy model...
Initial pipeline components: ['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner']
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmprh8ghzrm


100%|██████████| 492M/492M [00:16<00:00, 32.1MiB/s]


Finished download, copying /tmp/tmprh8ghzrm to cache at /root/.scispacy/datasets/2b79923846fb52e62d686f2db846392575c8eb5b732d9d26cd3ca9378c622d40.87bd52d0f0ee055c1e455ef54ba45149d188552f07991b765da256a1b512ca0b.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmp_0pvoo09


100%|██████████| 724M/724M [00:14<00:00, 53.6MiB/s]


Finished download, copying /tmp/tmp_0pvoo09 to cache at /root/.scispacy/datasets/7e8e091ec80370b87b1652f461eae9d926e543a403a69c1f0968f71157322c25.6d801a1e14867953e36258b0e19a23723ae84b0abd2a723bdd3574c3e0c873b4.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmpuhg6_ouc


...

Model loaded.

Extracting cancer entities from domain corpus...


Domain Corpus:   0%|          | 1/11642 [00:00<33:20,  5.82it/s]/usr/local/lib/python3.10/dist-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
Domain Corpus: 100%|██████████| 11642/11642 [03:36<00:00, 53.72it/s]


Computing term frequencies for reference corpus...


Reference Corpus: 100%|██████████| 1000/1000 [00:00<00:00, 2469.30it/s]


Identifying domain-specific concepts...
Computing embeddings...
Building taxonomy...
Depth limit reached for concept 'Primary Breast Cancer'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Breast Cancer Surgery'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Breast Cancer Database'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Breast Cancer Gene'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Black Breast Cancer'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Breast Cancers'. Adding under 'Lobular Breast Cancer'.
Depth limit reached for concept 'Breast Cancer Stem Cells'. Adding under 'Lobular Breast Cancer'.
Postprocessing taxonomy...
RDF Taxonomy saved to taxonomy.rdf
